In [9]:
workspace_id = "b353418e-13a0-4d6a-bc9d-7116bbe06c7e"          # Substitua pelo seu workspaceId
access_token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSIsImtpZCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSJ9.eyJhdWQiOiJodHRwczovL2FuYWx5c2lzLndpbmRvd3MubmV0L3Bvd2VyYmkvYXBpIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvOTNiYjBjMGQtYzkxNC00ZGJjLWEyYmQtMzFiZThjZDA2NGM5LyIsImlhdCI6MTc1NjgyNzM5MCwibmJmIjoxNzU2ODI3MzkwLCJleHAiOjE3NTY4MzEyOTAsImFpbyI6ImsyUmdZSkRiNlpCV3VGZDFaZTI1SDFOeXBDYXhBZ0E9IiwiYXBwaWQiOiIwNTRkODNjMC1lMmFlLTRkM2ItOTNmNC1kYTBiNDc1YmJhYjgiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC85M2JiMGMwZC1jOTE0LTRkYmMtYTJiZC0zMWJlOGNkMDY0YzkvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiJlMjUyN2I3Ni00MDkyLTRlNjUtYTg5OC0zYmU5Yjc2MDk1MjMiLCJyaCI6IjEuQVZvQURReTdreFRKdkUyaXZURy1qTkJreVFrQUFBQUFBQUFBd0FBQUFBQUFBQUJhQUFCYUFBLiIsInN1YiI6ImUyNTI3Yjc2LTQwOTItNGU2NS1hODk4LTNiZTliNzYwOTUyMyIsInRpZCI6IjkzYmIwYzBkLWM5MTQtNGRiYy1hMmJkLTMxYmU4Y2QwNjRjOSIsInV0aSI6InFzS1Q4LWU3S1U2Z3MwbzlhMDhFQUEiLCJ2ZXIiOiIxLjAiLCJ4bXNfZnRkIjoiZHpQbjY2MURiTFFDV0FqYjdkeFJCUHJsN3hEM0JaQWd0VnRJLWhoVUo2d0JkWE56YjNWMGFDMWtjMjF6IiwieG1zX2lkcmVsIjoiNyAxMiIsInhtc19yZCI6IjAuNDJMbFlCSmlqQklTNFdBWEV2Z3pMLUhNVDlzSlBrdnVfbG5HNzU3SEFSVGxGQktvWFJsYUxPako1REpCY0tOWXRyWFBJYUFvaDVBQUp3TUVISURTQUEifQ.UEoOk0NKh28NkIbFRX1lKUE7mdeBJdVCdMRUBvLHa20CVPQrsRZvdNGiz-FG2-CIGYdAIJRvR4MScUcWFH3TmyUP5umlNpqf7BZSvCY_5gedo7hQj_yCdK-rp6Dr146hFBWtGMtU7044f2lYlrZqUCaNQtxzA351BpkUIoqGAPZlqPvzTscu1sPm4lgcMQ-8ftKCSbLfqGDfDQkispc69d4FhFrqTRiLJJWt-OYDeXaN4FzEdVVwU8AQz5RSvaHHc_z-BulF4-hal6fz0kXxV3g2cn8S8JFruJHLUaZwfR8IFCybUitTbdjTxSbGHPJFgwg66v7-lVsaa5dy6yPIxA"          # Substitua pelo token válido
pbix_file_path = r"C:\Users\maria\Downloads\TesteDashbooster.pbix"  # Caminho do PBIX
dataset_name = "Dashbooster_API"           # Nome que você quer dar ao dataset


In [ ]:
import os
import requests
import sys


# -----------------------------
# 1️⃣ Verifica o arquivo PBIX
# -----------------------------
if not os.path.exists(pbix_file_path):
    print(f"Erro: arquivo PBIX não encontrado em {pbix_file_path}")
    sys.exit(1)

file_size = os.path.getsize(pbix_file_path)
if file_size > 1024**3:  # 1 GB
    print("Erro: PBIX maior que 1 GB, não é suportado via API REST")
    sys.exit(1)

print(f"Arquivo PBIX encontrado ({file_size / 1024**2:.2f} MB).")

# -----------------------------
# 2️⃣ Upload do PBIX
# -----------------------------
url_import = (
    f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/imports"
    f"?datasetDisplayName={dataset_name}"
)

headers = {"Authorization": f"Bearer {access_token}"}


with open(pbix_file_path, "rb") as f:
    files = {"file": f}
    print("Iniciando upload do PBIX...")
    response = requests.post(url_import, headers=headers, files=files)

if response.status_code not in [200, 202]:
    print("Erro no upload:", response.status_code, response.text)
    sys.exit(1)

import_data = response.json()
import_id = import_data.get("id")
print("Upload bem-sucedido! Import ID:", import_id)

# -----------------------------
# 3️⃣ Obter datasetId publicado
# -----------------------------
url_datasets = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/datasets"
response = requests.get(url_datasets, headers=headers)

if response.status_code != 200:
    print("Erro ao listar datasets:", response.status_code, response.text)
    sys.exit(1)

datasets = response.json().get("value", [])
dataset_id = None
for ds in datasets:
    if ds["name"] == dataset_name:
        dataset_id = ds["id"]
        break

if dataset_id:
    print("Dataset ID encontrado:", dataset_id)
else:
    print("Dataset não encontrado após o upload.")


Arquivo PBIX encontrado (4.96 MB).
Iniciando upload do PBIX...
Upload bem-sucedido! Import ID: 7e2bfd8e-1819-46c3-9fb3-76814d7aa2ae
Dataset não encontrado após o upload.


In [8]:
import requests

def listar_powerbi(workspace_id, access_token):
    """
    Lista os relatórios (Power BI) de um workspace.
    
    :param workspace_id: ID do workspace no Power BI
    :param access_token: Token de acesso OAuth2
    :return: Lista de dicionários com 'id' e 'name' dos relatórios
    """
    url_reports = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/reports"
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.get(url_reports, headers=headers)

    if response.status_code != 200:
        print("Erro ao listar relatórios:", response.status_code, response.text)
        return []

    reports = response.json().get("value", [])
    lista_reports = [{"id": r["id"], "name": r["name"]} for r in reports]

    return lista_reports

relatorios = listar_powerbi(workspace_id, access_token)
for r in relatorios:
    print(f"{r['name']} - {r['id']}")


Teste Dashbooster - 7868fb27-de86-4d0d-82f1-7aa571081455
Meu Relatório de Teste Clonado via API - 34064a14-d16d-4fd2-981a-4c9589b9f9ba
Meu Relatório de Teste Clonado via API - ba81cb49-d0e8-4d35-9c5c-e5ca0cb46ede
Meu Relatório de Teste Clonado via API - f50039f5-706f-4d50-8077-a7db22cd5173
MyReport - 557053af-90bb-4821-bede-8ed4c434adef
